# Existing magics

In [ ]:
%%html
<h1> Heading </h1>


In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R
x <- 42
print(x)

[1] 42


# Your new magics

In [ ]:
import re
from IPython.display import HTML
from IPython.core.magic import register_cell_magic

In [ ]:
def var_sub(m):
  v = globals()[m[1]]
  if type(v) == str:
    return v
  else:
    return repr(v)

In [ ]:
@register_cell_magic
def javascript(line="", cell=""):
  # var sub
  cell = re.sub(r'\{\{(\w+)+}}', var_sub, cell)

  m = re.match(r'\w+', line)
  if m:                       # assign to a variable
    globals()[m[0]] = cell
  else:                       # normal display
    display(HTML("""
<div id="drawing"></div>
<script src="https://cdnjs.cloudflare.com/ajax/libs/svg.js/2.6.5/svg.js"></script>
<script>
var draw = SVG('drawing').size('100%', 100)
var y_cursor_pos = 0
function print(x){
  draw.text(x.toString()).move(400, y_cursor_pos)
  y_cursor_pos += 20
}

text = s=> draw.text(s).font('anchor', 'middle')
line = (x1,y1, x2,y2)=> draw.line(x1,y1, x2, y2).stroke({width: 1})
max = Math.max
""" + cell))

In [ ]:
%%javascript
draw.rect(100, 100)

# Elasticsearch Magic

In [ ]:
# install
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [ ]:
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [ ]:
# wait a bit then test
!curl -X GET "localhost:9200/"

{
  "name" : "45bec8da0903",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "B-Ooi4-MRuaBI_pj95ntKQ",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [ ]:
from requests import models, Session, Request
from urllib.parse import urljoin
from IPython.core.magic import register_cell_magic
import re

# to display json response
def render(r):
  text = r.text
  if text[0] in "[{":
    return """
    <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
    <script>
      renderjson.set_show_to_level(1)
      document.body.appendChild(renderjson(%s))
      new ResizeObserver(google.colab.output.resizeIframeToContent).observe(
          document.body)
    </script>
    """ % text
  else:
    return "<pre>%s</pre>" % text
models.Response.repr_html_ = render

In [ ]:
@register_cell_magic
def es(line="", cell=""):
  cell = re.sub(r'(?m)^\s*#.*\n?','', cell)
  line1 = (cell + '\n').find('\n')
  method, path = cell[:line1].split(None, 1)
  body = cell[line1:].strip()
  args = {}
  if body:
    args['data'] = (body + '\n').encode()
  rsp = Session().send(Request(method, urljoin('http://localhost:9200', path), **args).prepare())
  return rsp

In [ ]:
%%es
GET /

<Response [200]>

In [ ]:
%%es
PUT /customer/_doc/1
{
    "name" : "John Doe"
}

<Response [406]>

In [ ]:
%%es
PUT /customer/_doc/2
{
    "name" : "John smith"
}

<Response [406]>

#Python calls JS

In [ ]:
%%html
<script>
  document.body.appendChild(document.createTextNode("Hello World"))
</script>

In [ ]:
from IPython.display import Javascript
Javascript('document.body.append(document.createTextNode("Hello"))')

<IPython.core.display.Javascript object>

In [ ]:
from google.colab.output import eval_js
eval_js('1+1')

2

#JS calls Python

#Toy Example

In [ ]:
from IPython.display import JSON
import IPython.display as idisplay
from google.colab import output

def concat(a, b):
  return JSON({'result': '%s %s' % (a, b)})

output.register_callback('concat', concat)

In [ ]:
idisplay.Javascript('''
(async function() {
  const result = await google.colab.kernel.invokeFunction(
    'notebook.concat',
    ['hello', 'world'],
    {});

    const text = result.data['application/json'].result;
    document.querySelector("output-area").appendChild(document.createTextNode(text))
})();
''')

<IPython.core.display.Javascript object>

#Take a photo

In [ ]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

In [ ]:
VIDEO_HTML = """
<video autoplay
width=800 height=600></video>
<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  display(HTML(VIDEO_HTML % quality))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)

In [ ]:
take_photo()

46083